In [246]:
import pandas as pd
import numpy as np
import networkx as nx
import os
from os import listdir
import random

# Constructing Gene Sets Using Shortest Path from an AMR gene to conserved gene

This following steps allow to construct gene sets to run the model from Nguyen et. al. based on shortest path of conserved genes obtained considering the distance from a conserved gene to an AMR gene. We are working with the following subspecies choosed to have the same RefSeq on PATRIC database and STRING:

- Klebsiella pneumoniae subsp. pneumoniae MGH 78578
- Mycobacterium tuberculosis H37Rv
- Salmonella enterica subsp. enterica serovar Typhimurium str. LT2
- Staphylococcus aureus subsp. aureus NCTC 8325

In [247]:
bacteria = ['Mycobacterium', 'Klebsiella', 'Salmonella', 'Staphylococcus']

To construct gene sets for a specific bacteria, modify the index below according to the list above

In [252]:
root = os.getcwd() + '\\' + bacteria[2] 

# Data From PATRIC

## Genomic Features

The table below contains a list of genomic features, including coding DNA.

Each feature is identified by BRC ID from PATRIC and associated to a protein family referred as PATRIC genus-specific families (PLfams). Most of the genes has the associated Protein ID.

In [253]:
features = pd.read_csv(f'{root}\\genome_features.csv')

In [254]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4970 entries, 0 to 4969
Data columns (total 21 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Genome                                   4970 non-null   object 
 1   Genome ID                                4970 non-null   float64
 2   Accession                                4970 non-null   object 
 3   BRC ID                                   4970 non-null   object 
 4   RefSeq Locus Tag                         4460 non-null   object 
 5   Alt Locus Tag                            4970 non-null   object 
 6   Feature ID                               4970 non-null   object 
 7   Annotation                               4970 non-null   object 
 8   Feature Type                             4970 non-null   object 
 9   Start                                    4970 non-null   int64  
 10  End                                      4970 no

Through this table, we extract useful data to map protein families referred by Nguyen et. al.:

In [255]:
plf = features[['BRC ID', 'PATRIC genus-specific families (PLfams)']].astype("string")
plf.columns = ['BRC_ID', 'PLFam']
plf.set_index('BRC_ID', inplace = True)
plf

,PLFam
BRC_ID,
fig|99287.12.peg.977,PLF_590_00000012
fig|99287.12.peg.978,PLF_590_00014201
fig|99287.12.peg.979,PLF_590_00019426
fig|99287.12.peg.980,PLF_590_00015851
fig|99287.12.peg.981,PLF_590_00000025
...,...
fig|99287.12.peg.4975,PLF_590_00006047
fig|99287.12.peg.4976,PLF_590_00005031
fig|99287.12.peg.4844,PLF_590_00005081


## Specialty Genes

A table containing AMR genes for this specie according to PATRIC.

In [256]:
AMR_refseq = pd.DataFrame(pd.read_csv(f'{root}\\specialty_genes.csv')['RefSeq Locus Tag'])
AMR_refseq.columns = ['AMR_RefSeq']

## Data from Nguyen et. al.

Be sure you have data from Nguyen et. al inside a directory of the specific bacteria 

In [257]:
plf_500 = []

datadir = root + '\\Nguyen_et_al_2020\\fasta.500.0'
for strain in listdir(datadir):
    with open(os.path.join(datadir, strain), 'r') as sequences:
        for line in sequences:
            if line[0] == '>':
                plf_500.append(line[1:len(line)-1])
                
plf_500 = pd.DataFrame(np.unique(plf_500))
plf_500.columns = ['Protein Family ID']

## RefSeq Mapping

In [258]:
refseq = features[['BRC ID', 'RefSeq Locus Tag']]
refseq.columns = ['BRC_ID', 'RefSeq']
refseq.set_index('BRC_ID', inplace = True)

In [259]:
plf_map_refseq = features[['RefSeq Locus Tag', 'PATRIC genus-specific families (PLfams)']][features['PATRIC genus-specific families (PLfams)'].isin(plf_500['Protein Family ID'])].reset_index(drop = True)
plf_map_refseq.columns = ['RefSeq', 'PLF']
plf_map_refseq.dropna(inplace = True)
plf_map_refseq.reset_index(drop = True, inplace = True)

In [260]:
plf_map_refseq

,RefSeq,PLF
0,STM0934,PLF_590_00001917
1,STM0936,PLF_590_00003070
2,STM0942,PLF_590_00000215
3,STM0961,PLF_590_00001933
4,STM0969,PLF_590_00001385
...,...,...
487,STM0869,PLF_590_00002654
488,STM0874,PLF_590_00000196
489,STM0876,PLF_590_00001271
490,STM0889,PLF_590_00000747


## Protein Interaction Network

In [261]:
ppi_patric = pd.read_csv(f'{root}\\ppi_patric.csv')
ppi_patric = ppi_patric[['Interactor A ID', 'Interactor B ID']].astype("string")
ppi_patric.columns = ['Interactor_A_ID', 'Interactor_B_ID']

In [262]:
ppi_refseq = ppi_patric
for i in range(len(ppi_refseq['Interactor_A_ID'])):
    if ppi_refseq['Interactor_A_ID'][i] in list(refseq.index):
        ppi_refseq.at[i, 'Interactor_A_ID'] = refseq.loc[ppi_refseq['Interactor_A_ID'][i]].RefSeq
    else:
        ppi_refseq.drop(inplace = True, labels = i)
        
ppi_refseq.reset_index(inplace = True, drop = True)
        
for i in range(len(ppi_refseq['Interactor_B_ID'])):
    if ppi_refseq['Interactor_B_ID'][i]in (refseq.index):
        ppi_refseq.at[i, 'Interactor_B_ID'] = refseq.loc[ppi_refseq['Interactor_B_ID'][i]].RefSeq
    else:
        ppi_refseq.drop(inplace = True, labels = i)

In [263]:
ppi_string = pd.read_csv(f'{root}\\ppi_string.txt', sep = ' ')
ppi_string.columns = ['Interactor_A_ID', 'Interactor_B_ID', 'weight']
ppi_string

for i in ppi_string.index:
    ppi_string['Interactor_A_ID'][i] = ppi_string['Interactor_A_ID'][i].split('.')[1]
    ppi_string['Interactor_B_ID'][i] = ppi_string['Interactor_B_ID'][i].split('.')[1]
ppi_string

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Interactor_A_ID,Interactor_B_ID,weight
0,STM0002,STM0003,323
1,STM0003,STM0002,323
2,STM0002,STM0067,411
3,STM0067,STM0002,411
4,STM0002,STM0097,436
...,...,...,...
105069,STM4585,STM4578,403
105070,STM4588,STM4589,627
105071,STM4589,STM4588,627
105072,STM4592,STM4593,173


In [264]:
ppi = pd.DataFrame(pd.concat([ppi_refseq, ppi_string], axis = 0)[['Interactor_A_ID', 'Interactor_B_ID']]).reset_index(drop = True)

In [1]:
ppi

NameError: name 'ppi' is not defined

In [265]:
conserved_ppi_A = plf_map_refseq[plf_map_refseq['RefSeq'].isin(ppi['Interactor_A_ID'])]['RefSeq']
conserved_ppi_B = plf_map_refseq[plf_map_refseq['RefSeq'].isin(ppi['Interactor_B_ID'])]['RefSeq']

conserved_ppi = pd.DataFrame(pd.concat([conserved_ppi_A, conserved_ppi_B], axis = 0).drop_duplicates())

In [266]:
AMR_ppi_A = AMR_refseq[AMR_refseq['AMR_RefSeq'].isin(ppi['Interactor_A_ID'])]['AMR_RefSeq']
AMR_ppi_B = AMR_refseq[AMR_refseq['AMR_RefSeq'].isin(ppi['Interactor_B_ID'])]['AMR_RefSeq']

AMR_ppi = pd.DataFrame(pd.concat([AMR_ppi_A, AMR_ppi_B], axis = 0))
AMR_ppi.reset_index(drop=True, inplace=True)

In [267]:
AMR_ppi['AMR_RefSeq'].nunique()

66

In [268]:
AMR_ppi.drop_duplicates(inplace = True)

In [269]:
ppi_info = pd.DataFrame(columns = ['Conserved Gene', 'Shortest Path to an AMR gene (length)',])

ppi_info['Conserved Gene'] = conserved_ppi.reset_index(drop = True)['RefSeq']

In [270]:
ppi_graph = nx.from_pandas_edgelist(ppi, 'Interactor_A_ID', 'Interactor_B_ID')

In [271]:
ppi_graph.number_of_edges()

56279

In [272]:
ppi_graph.number_of_nodes()

3946

In [274]:
idx = 0
for i in conserved_ppi['RefSeq']:
    lengths = []
    for j in AMR_ppi['AMR_RefSeq']:
        if nx.has_path(ppi_graph, i, j):
            lengths.append(nx.shortest_path_length(ppi_graph, i, j))
    if lengths:        
        ppi_info['Shortest Path to an AMR gene (length)'][idx] = min(lengths)
        
    idx += 1

In [275]:
plf_map_refseq = plf_map_refseq.set_index('RefSeq')

In [276]:
for i in ppi_info['Shortest Path to an AMR gene (length)'].unique():
    gene_set = ppi_info[ppi_info['Shortest Path to an AMR gene (length)'] == i]['Conserved Gene']
    keys = plf_map_refseq.index.intersection(gene_set)
    globals()[f'plf_length_{i}'] = plf_map_refseq.loc[list(keys)]['PLF']

In [281]:
bacdir = f'{root}\\GeneSets\\PathLengths'

os.mkdir(bacdir)

for i in ppi_info['Shortest Path to an AMR gene (length)'].unique():
    newdir = f'Length_{i}'
    lendir = os.path.join(bacdir, newdir)
    os.mkdir(lendir)
    
    sample = f'{root}\\Nguyen_et_al_2020\\fasta.500.0'
    for strain in listdir(sample):
        with open(os.path.join(lendir, strain), 'a') as mystrain:
            with open(os.path.join(sample, strain), 'r') as sequences:
                first_loop = True
                for line in sequences:
                    if line[0] == '>':
                        if first_loop:
                            plfam = line[1:len(line)-1]
                            seq = ''
                            first_loop = False
                            continue
                        if plfam in list((globals()[f'plf_length_{i}'])):
                            mystrain.write('>' + plfam + '\n')
                            mystrain.write(seq)
                        plfam = line[1:len(line)-1]
                        seq = ''
                    else:
                        seq += line
                if plfam in list((globals()[f'plf_length_{i}'])):
                            mystrain.write('>' + plfam + '\n')
                            mystrain.write(seq)

At this point, it is possible to use this new configuration of data to run the model

Note we have 10 genes with length zero. This means that there are 10 conserved genes in the ppi annotated as AMR genes today, but they were not when the data from paper was collected. let's make groups of 10 genes to compare with the group of 10 AMR genes.

In [277]:
print(ppi_info.groupby(['Shortest Path to an AMR gene (length)']).size().reset_index(name='Count'))

   Shortest Path to an AMR gene (length)  Count
0                                      0     10
1                                      1    145
2                                      2    261
3                                      3     38
4                                      4      4


Let's take as many non overlapping groups of 10 genes as we can:

In [307]:
dir_genes = os.path.join(bacdir, '10genes')
os.mkdir(dir_genes)

num_genes = [10, 140, 260, 30]

for i in range(4):
    for j in range(1, int(num_genes[i]/10)+2):
        repdir = os.path.join(dir_genes, f'length.{i}.{j}')
        os.mkdir(repdir)

In [308]:
for i in [1, 2, 3]:
    j = 1
    lendir = os.path.join(bacdir, f'Length_{i}')
    for strain in listdir(lendir):
        with open(os.path.join(lendir, strain), 'r') as sequences:
            repdir = os.path.join(dir_genes, f'length.{i}.{j}')
            c = 0
            mystrain = open(os.path.join(repdir, strain), 'a')
            first_loop = True        
            for line in sequences:
                if line[0] == '>':
                    if first_loop:
                        plfam = line
                        seq = ''
                        first_loop = False
                        continue
                    mystrain.write(plfam)
                    mystrain.write(seq)
                    plfam = line
                    seq = ''
                    c += 1
                else:
                    seq += line
                if c == 10:
                    mystrain.close()
                    j += 1
                    repdir = os.path.join(dir_genes, f'length.{i}.{j}')
                    c = 0
                    mystrain = open(os.path.join(repdir, strain), 'a')
            j = 1
            mystrain.close()